In [1]:
!pip install transformers

# basic
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
import random
import time
import datetime
import numpy as np
import os
import tqdm
%config InlineBackend.figure_format = 'retina'
import shutil
import logging
logging.basicConfig(level=logging.ERROR)

    # # tf
    # from tensorflow.keras.datasets import reuters
    # from tensorflow.keras.models import Sequential
    # from tensorflow.keras.layers import Dense, LSTM, Embedding
    # from tensorflow.keras.preprocessing.sequence import pad_sequences
    # from tensorflow.keras.utils import to_categorical
    # from tensorflow.keras.models import load_model
    # from tensorflow.keras.preprocessing.text import Tokenizer

#torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# transformers(BERT)
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 19.3MB/s 
     |████████████████████████████████| 1.1MB 47.2MB/s 
     |████████████████████████████████| 890kB 67.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bd32667bcf8fc314cbe1415f1e191fa2a0d3848138c783e08c96c01ad60f26dc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
bert = BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [3]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
train = pd.read_csv("/content/drive/My Drive/bluehouse/train.csv", usecols=['category','data'])[['data','category']].dropna()
test = pd.read_csv('/content/drive/My Drive/bluehouse/test.csv', encoding = 'utf-8')

In [5]:
train_sentences = train.data.values
train_labels = train.category.values

train_input_ids = []
train_attention_masks = []

for sent in train_sentences:
    encoded_dict = tokenizer.encode_plus(
                sent,
                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                max_length = 512,           # Pad & truncate all sentences.
                pad_to_max_length = True,
                return_attention_mask = True,   # Construct attn. masks.
                return_tensors = 'pt')  
    train_input_ids.append(encoded_dict['input_ids'])
    train_attention_masks.append(encoded_dict['attention_mask'])

train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

In [6]:
test_sentences = test.data.values

test_input_ids = []
test_attention_masks = []

for sent in test_sentences:
    encoded_dict = tokenizer.encode_plus(
                sent,
                add_special_tokens = True, 
                max_length = 512,           
                pad_to_max_length = True,
                return_attention_mask = True,  
                return_tensors = 'pt')   
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

In [7]:
dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
prediction_data = TensorDataset(test_input_ids, test_attention_masks)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(len(prediction_data)))

35,992 training samples
4,000 validation samples
5,000 test samples


In [8]:
train_batch_size = 15
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = train_batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = train_batch_size # Evaluate with this batch size.
        )

test_batch_size = 15
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=test_batch_size)

In [9]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [10]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [11]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-6, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
# Number of training epochs. The BERT authors recommend between 2 and 4. 
epochs = 8

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [12]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [13]:
check_path = '/content/drive/My Drive/bluehouse/chp/current_checkpoint.pt'
best_path = '/content/drive/My Drive/bluehouse/chp/best/best_checkpoint.pt'

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)

def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

In [ ]:
seed_val = 0

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
valid_loss_min = np.inf
total_t0 = time.time()


for epoch_i in tqdm.notebook.tqdm(range(0, epochs)):
    #               Training

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in tqdm.notebook.tqdm(enumerate(train_dataloader)):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)


        model.zero_grad()        

        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        total_train_loss += loss.item()

        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step() # Update the learning rate.

    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

    # Validation

    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in tqdm.notebook.tqdm(validation_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid_Loss': avg_val_loss,
            'Valid_Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    save_ckp(training_stats, False, check_path, best_path)

    if avg_val_loss <= valid_loss_min: # if bset model
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,avg_val_loss))
        save_ckp(training_stats, True, check_path, best_path)
        valid_loss_min = avg_val_loss

    # test
    print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

    model.eval()
    predictions = []

    for batch in tqdm.notebook.tqdm(prediction_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,  # Telling the model not to compute or store gradients, saving memory and 
                            attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    print('DONE.')

    flat_predictions = np.concatenate(predictions, axis=0)
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

    submission = pd.DataFrame(flat_predictions)
    submission.columns = ['category']
    submission.index.name = 'index'
    submission.to_csv('/content/drive/My Drive/bluehouse/BH_Batch8_Maxlen512_epoch4_lr4e-5_{}.csv'.format(epoch_i+1), encoding='utf-8')


======== Epoch 1 / 8 ========
Training...


  Batch    50  of  2,400.    Elapsed: 0:00:40.
  Batch   100  of  2,400.    Elapsed: 0:01:20.
  Batch   150  of  2,400.    Elapsed: 0:01:59.
  Batch   200  of  2,400.    Elapsed: 0:02:39.
  Batch   250  of  2,400.    Elapsed: 0:03:18.
  Batch   300  of  2,400.    Elapsed: 0:03:58.
  Batch   350  of  2,400.    Elapsed: 0:04:37.
  Batch   400  of  2,400.    Elapsed: 0:05:17.
  Batch   450  of  2,400.    Elapsed: 0:05:57.
  Batch   500  of  2,400.    Elapsed: 0:06:36.
  Batch   550  of  2,400.    Elapsed: 0:07:16.
  Batch   600  of  2,400.    Elapsed: 0:07:55.
  Batch   650  of  2,400.    Elapsed: 0:08:35.
  Batch   700  of  2,400.    Elapsed: 0:09:14.
  Batch   750  of  2,400.    Elapsed: 0:09:54.
  Batch   800  of  2,400.    Elapsed: 0:10:33.
  Batch   850  of  2,400.    Elapsed: 0:11:13.
  Batch   900  of  2,400.    Elapsed: 0:11:53.
  Batch   950  of  2,400.    Elapsed: 0:12:32.
  Batch 1,000  of  2,400.    Elapsed: 0:13:12.
  Batch 1,050  of  2,400.    Elapsed: 0:13:51.
  Batch 1,100

In [ ]:
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid_Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()
